In [1]:
import sys
import pylab
import random
import numpy as np
from collections import deque
import environment

import torch
from torch import nn, optim
import torch.nn.functional as F

from requests import get

In [2]:
# 카트폴 예제에서의 DQN 에이전트
class DQNAgent:
    def __init__(self, state_size, action_size):
        '''
        구글 colab에서는 아래 render를 True로 만들면 실행이 안됩니다.
        '''
        self.render = True

        '''
        저장해 놓은 신경망 모델을 가져올 지 선택합니다. (mountaincar_trainded.h5)
        훈련을 중간에 중단해 놓았다가 다시 시작하려면 아래를 True로 바꾸고 실행하시면 됩니다.
        '''
        self.load_model = True

        # 상태와 행동의 크기 정의
        self.state_size = state_size
        self.action_size = action_size

        # DQN 하이퍼파라미터
        '''
        일단 None이라고 되어있는 부분 위주로 수정해주세요. (다른 것들 잘못 건드시면 안될수도 있음)
        아래 8개 하이퍼파라미터(maxlen 포함)는 cartpole_dqn 예제 그대로 복사하셔도 되고, 좀 수정하셔도 됩니다.
        '''
        self.discount_factor = 0.99
        self.learning_rate = 0.01
        self.epsilon = 1.0
        # self.epsilon_decay = 0.1
        # self.epsilon_min = 0.10
        self.batch_size = 64
        self.train_start = 100

        # 리플레이 메모리, 최대 크기 10000
        self.memory = deque(maxlen=2000)

        self.action_buffer = []
        
        # 모델과 타깃 모델 생성
        '''
        아마 그냥 실행하시면 오류가 날텐데
        build_model을 완성하시면 오류가 사라집니다.
        '''
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.optimizer = optim.Adam(
            self.model.parameters(), lr=self.learning_rate)

        # 타깃 모델 초기화
        self.update_target_model()

        if self.load_model:
            self.model.load_state_dict(torch.load(
                './save_model/tank_dqn_14.bin'))

    # 상태가 입력, 큐함수가 출력인 인공신경망 생성
    def build_model(self):
        model = nn.Sequential(
            nn.Linear(self.state_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, self.action_size),
        )
        
        return model

    # 타깃 모델을 모델의 가중치로 업데이트
    def update_target_model(self):
        self.target_model.load_state_dict(self.model.state_dict())

    # 입실론 탐욕 정책으로 행동 선택
    def get_action(self, state):
        # 버퍼에 남으면 버퍼 수행
        if len(self.action_buffer) > 0:
            print('buffer', self.action_buffer)
            return torch.LongTensor([[self.action_buffer.pop()]])

        # 휴리스틱 가능하면 휴리스틱
        buffer = env.try_to_kill()
        if buffer:
            buffer.reverse()
            print('buffer_first', buffer)
            self.action_buffer = buffer
            return torch.LongTensor([[self.action_buffer.pop()]])
        
        # 아니면 랜덤
        else:
            print('random')
            # 무작위 행동 반환
            legal_actions = env.legal_actions()
            actions = []
            if 3 in legal_actions:
                actions.append(3)
            if 4 in legal_actions:
                actions.append(4)
            if 5 in legal_actions:
                actions.append(5)
            if 6 in legal_actions:
                actions.append(6)
            if 7 in legal_actions: # 랜덤으로 종료는 안나옴
                actions.append(7)
            return torch.LongTensor([[random.choice(actions)]])

    # 샘플 <s, a, r, s'>을 리플레이 메모리에 저장
    def append_sample(self, state, action, reward, next_state, done):
        reward = torch.FloatTensor([reward])
        next_state = torch.FloatTensor([next_state])
        done = torch.FloatTensor([done])

        self.memory.append((state, action, reward, next_state, done))

    # 리플레이 메모리에서 무작위로 추출한 배치로 모델 학습
    def train_model(self):
        # 메모리에서 배치 크기만큼 무작위로 샘플 추출
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.cat(states)
        actions = torch.cat(actions)
        rewards = torch.cat(rewards)
        next_states = torch.cat(next_states)
        dones = torch.cat(dones)

        # 현재 상태에 대한 모델의 큐함수
        # 다음 상태에 대한 타깃 모델의 큐함수
        current_q = self.model(states).gather(1, actions-3)
        max_next_q = self.target_model(next_states).detach().max(1)[0]
        expected_q = rewards + (self.discount_factor * max_next_q)

        # 벨만 최적 방정식을 이용한 업데이트 타깃
        self.optimizer.zero_grad()

        loss = F.mse_loss(current_q.squeeze(), expected_q)
        loss.backward()

        self.optimizer.step()


In [3]:
try:
    del env
except:
    pass
env = environment.Environment()

state_size = len(env._get_state())
action_size = 5 # 이동, 종료만 인정

agent = DQNAgent(state_size, action_size)

scores, episodes, epsilons, max_poss = [], [], [], []

In [5]:
key = '0d04e6138db8a6a574ecfacfb55a98160b9b3ec6348f86ef122d2bbc66ae397b'

In [7]:
done = False
score = 0
# env.start(ip=ip) # resource + create
env.tankAPI.ip = '192.168.0.5'
env.tankAPI.key = key
env.reset() # reset
env.tankAPI.session_join() # join

join DONE
join..done!


In [9]:
env.render()

--------------------------------------------------------------
O O O · O O O O O O O O O · ·                                   
O O O · O O O O O O O O O O ·                                   
O · O O O · O O O O O O O O ·         · · · · · · O O · · · ·   
O · O O O · · · · T O O O O ·         · O O · · · O O O O · ·   
O · O O O O O · · · · · · O ·         · O O · · · · · O O · ·   
O T · · · O O · · · · · · · ·         O O O · · O O O O O O ·   
O · O O · O O · · O O O O · · · · · · · O O · · O O O O O O ·   
O · O O · · · · · O O O · · · · · · · O O O O · · O O O O O ·   
O · O O · · · · · · · · · · · · O O O O O · · · · · O O O O ·   
O · · · · · · · · · · · · · O O O O O O O · · · · · O O O O ·   
· · · · O O · · · ·         O O O O O O · · · · · · · · · · ·   
· · O · O · · · · ·         · · · · · · · · · · · · · · · · ·   
  · O · ·       · ·         · · · · · · · · · · · · · · · · ·   
O · · · · · ·   · ·         · · · · · · · · O O O · · · · · ·   
O · · · · · · · · · · · · ·

In [11]:
state = env.reset2([0, 0, 0, 0]) # input: 탱크의 포각들 (45의 배수)
env.turn_tank = 1

while not done:
    env.render()
    
    state = torch.FloatTensor([state])
    action = agent.get_action(state)
    
    next_state, reward, done, info = env.step(action)
    if info['action'] == 7:
        agent.action_buffer.clear()
        print('buffer_clear')
    
    state = next_state
    
    if done:
        agent.update_target_model()
        
        print(info)
        print('\n','*'*30,'\n')


s!s!v!!!!--------------------------------------------------------------
O O O · O O O O O O O O O · ·                                   
O O O · O O O O O O O O O O ·                     O             
O · O O O · O O O O O O O O ·         · · · · · · O O · · · ·   
O · O O O · · · · 2 O O O O ·         · O O · · · O O O O · ·   
O · O O O O O · · · · · · O ·         · O O · · · · · O O · ·   
O 1 · · · O O · · · · · · · · · · O O O O O · · O O O O O O ·   
O · O O · O O · · O O O O · · O · · · · O O · · O O O O O O ·   
O · O O · · · · · O O O · · · · · · · O O O O · 4 O O O O O ·   
O · O O · · · · · · · · · · O · O O O O O · · · · · O O O O ·   
O · · · · · · · · · · · · · O O O O O O O · · · · · O O O O ·   
· · · · O O · · · ·         O O O O O 3 · · · · · · · · · · ·   
O · O · O O · O · ·         · · · · · · · · · · · · · · · · ·   
O · O · · · · O · ·         · · · · · · · · · · · · · · · · ·   
O · · · · · · · · · · · ·   · · · · · · · · O O O · · · · · ·   
O · · · · · · · · 

KeyboardInterrupt: 